# Analyse af COVID-19 data

## Del 1: indlæs data
0. Importere `pandas` og brug pandas.read_csv(path) til at hente data direkte fra GitHub-linket.

    (https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/05-26-2020.csv)

In [3]:
import pandas as pd

date = '05-26-2020'
path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/05-26-2020.csv'

df = pd.read_csv(path)

1. Skimmelæs doc for bibliotek [datetime](https://docs.python.org/3/library/datetime.html)
2. Brug `datetime` til at specificere en dato.
3. Konstruér fil-stien ud fra den dato og hente dataen.
4. Brug `df.head()` og `df.info()` for at få overblik af dataframe. Find ud af, hvilke kolonner der findes (fx Country_Region, Confirmed, Deaths, Recovered).

## Del 2: datarensning

Læs artiklen [Pythonic Data Cleaning With pandas and NumPy](https://realpython.com/python-data-cleaning-numpy-pandas/)
Rense og konstruér dataframe `df`, der indholde kolonerne "Country", "Latitude", "Longitude", "Confirmed", "Deaths", "Recovered" og "Active". 

## Del 3: gruppér data
1. Gruppér evt. efter "Country" og summér antal Confirmed og Deaths.
2. Find de 10 lande med flest smittede den dag.

## Del 4: Visualisering
Vælg noget plot eksampler fra [seaborn](https://seaborn.pydata.org/examples/index.html) og 
1. Lav et bar plot over de 10 lande med flest smittede.
2. Lav et pie chart over fordelingen af dødsfald mellem de samme 10 lande.

## Ekstra udfordring: Kortvisualisering med Plotly
**Plotly** er et Python-bibliotek til interaktive visualiseringer, der kan bruges direkte i Jupyter Notebooks, Python scripts eller integreres i webapps. Det kan lave alt fra simple linjediagrammer til avancerede 3D-grafer og kort.

**Plotly Express** er en “hurtig” version af **Plotly**, der gør det nemt at lave grafer med få linjer kode. Det minder meget om seaborn eller pandas.plot, men med interaktivitet. 

Kig lidt ind i https://plotly.com/python/plotly-express/ og bruge `plotly.express.choropleth` til at lave et interaktivt verdenskort, hvor landene farves efter antallet af bekræftede COVID-19-tilfælde (eller dødsfald) på en given dato.